In [ ]:
# default_exp bbox_canvas

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from nbdev import *

In [ ]:
# exporti
import numpy as np
import traitlets
import ipywidgets as widgets
from copy import deepcopy
from pathlib import Path
from math import log
from ipycanvas import MultiCanvas, hold_canvas
from ipywidgets import Image, Label, Layout, HBox, VBox, Output

# Draw Box an Canvas

In [ ]:
from ipycanvas import MultiCanvas

# Create a multi-layer canvas with 4 layers
multi_canvas = MultiCanvas(4, width=200, height=200)
multi_canvas[0] #  Access first layer (background)
multi_canvas[3] #  Access last layer
multi_canvas

In [ ]:
# dynamic update of canvas size
multi_canvas.width = 30
multi_canvas.height = 60

understand canvas grid

![grid illustration](https://ipycanvas.readthedocs.io/en/latest/_images/grid.png)

In [ ]:
#exporti

def draw_bg(canvas, color='rgb(236,240,241)'):
    with hold_canvas(canvas):
        canvas.fill_style = color
        canvas.fill_rect(0, 0, canvas.size[0], canvas.size[1])

In [ ]:
from ipycanvas import Canvas
canvas = Canvas(width=300, height=20)
draw_bg(canvas)
canvas

In [ ]:
canvas = Canvas(width=200, height=200)
canvas.fill_rect(25, 25, 100, 100)
canvas.clear_rect(45, 45, 60, 60)
canvas.stroke_rect(50, 50, 50, 50)
canvas

In [ ]:
#exporti

def draw_bounding_box(canvas, coords, color='white', line_width=None,
                      border_ratio=2, clear=False):
    with hold_canvas(canvas):
        if clear:
            canvas.clear()

        pos_x, pos_y, rect_x, rect_y = coords
        
        canvas.line_width = line_width or log(canvas.height)/5
        gap = canvas.line_width * border_ratio
        
        canvas.fill_style = color
        
        canvas.stroke_rect(pos_x, pos_y, rect_x, rect_y)
        canvas.fill_rect(pos_x, pos_y, rect_x, rect_y)
        canvas.stroke_rect(pos_x, pos_y, rect_x, rect_y)
        canvas.clear_rect(pos_x + gap, pos_y + gap, rect_x - 2 * gap,
                          rect_y - 2 * gap)
        canvas.stroke_rect(pos_x + gap, pos_y + gap, rect_x - 2 * gap,
                           rect_y - 2 * gap)

In [ ]:
canvas = Canvas(width=100, height=100)
draw_bg(canvas)
draw_bounding_box(canvas, [10, 0, 80, 100])
draw_bounding_box(canvas, np.array([0, 40, 80, 100]) / 2, color='red', border_ratio=4, line_width=1.5)
canvas

In [ ]:
canvas = MultiCanvas(2, width=200, height=200)
draw_bg(canvas[0])
draw_bounding_box(canvas[1], [10, 40, 80, 100])
draw_bounding_box(canvas[1], np.array([10, 40, 80, 100]) / 2, color='red', border_ratio=4)
canvas

# Draw Image

- draw on canvas
- canvas with bg
- get image boundaries on canvas
- get image scaling to retrieve org coordinates

https://ipycanvas.readthedocs.io/en/latest/drawing_images.html

In [ ]:
from ipywidgets import Image

from ipycanvas import Canvas
from pathlib import Path

examples_path = Path("../data/projects/bbox/pics")
print(examples_path / "test200x200.png")
# sprite1 = Image.from_file(examples_path/"test200x200.png")

sprite1 = Image.from_file("../data/projects/bbox/pics/test200x200.png")
sprite2 = Image.from_file("../data/projects/bbox/pics/red400x640.png")

canvas = Canvas(width=300, height=300)

canvas.fill_style = '#a9cafc'
canvas.fill_rect(0, 0, 300, 300)

canvas.draw_image(sprite1, 50, 50)
canvas.draw_image(sprite2, 100, 100, width=40, height=40)

canvas

You can draw from another Canvas widget. This is the fastest way of drawing an image on the canvas.

In [ ]:
canvas2 = Canvas(width=600, height=300)
# Here ``canvas`` is the canvas from the previous example
canvas2.draw_image(canvas, 0, 0)
canvas2.draw_image(canvas, 300, 0)
canvas2

In [ ]:
#exporti
from PIL import Image as pilImage

# can we do this without reading image?
def get_image_size(path):
    pil_im = pilImage.open(path)
    return pil_im.width, pil_im.height

In [ ]:
get_image_size("../data/projects/bbox/pics/red400x640.png")

## Draw resized Image

- resize 

In [ ]:
#exporti

def draw_img(canvas, file, clear=False):
    # draws resized image on canvas and returns scale used
    with hold_canvas(canvas):
        if clear:
            canvas.clear()

        sprite1 = Image.from_file(file)

        width_canvas, height_canvas = canvas.width, canvas.height
        width_img, height_img = get_image_size(file)

        ratio_canvas = float(width_canvas) / height_canvas
        ratio_img = float(width_img) / height_img

        if ratio_img > ratio_canvas:
            # wider then canvas, scale to canvas width
            scale = width_canvas / width_img
        else:
            # taller then canvas, scale to canvas hight
            scale = height_canvas / height_img

        canvas.draw_image(sprite1, 0, 0, 
                          width=width_img * min(1, scale),
                          height=height_img * min(1, scale))
        return scale

In [ ]:
file = "../data/projects/bbox/pics/red400x640.png"
canvas = Canvas(width=300, height=300)
draw_bg(canvas)
scale = draw_img(canvas, file)
print(scale)
canvas

In [ ]:
file = "../data/projects/bbox/pics/green640x400.png"
canvas = Canvas(width=300, height=300)
draw_bg(canvas)
scale = draw_img(canvas, file)
print(scale)
canvas

## Catch Drawing Events

drawing sequence:

- mouse press
- mouse move
- mouse release

only watching on canvas area covered by image

In [ ]:
file = "../data/projects/bbox/pics/green640x400.png"
canvas = Canvas(width=300, height=300)
draw_bg(canvas)
scale = draw_img(canvas, file)
print(scale)

In [ ]:
debug_view = widgets.Output(layout={'border': '1px solid black'})

state = {'active': False}

@debug_view.capture(clear_output=True)
def show_state():
    print(state)

@debug_view.capture(clear_output=True)
def update_start(x, y):
    print('start')
    state['active'] = True
    state['start'] = (x, y)
    # reset end
    state['end'] = state['start']
    show_state()

@debug_view.capture(clear_output=True)
def update_state(x, y):
    if state['active']:
        state['end'] = (x, y)
        show_state()
        
@debug_view.capture(clear_output=True)
def update_end(x, y):
    state['active'] = False
    state['end'] = (x, y)
    show_state()
        
canvas.on_mouse_down(update_start)
canvas.on_mouse_move(update_state)
canvas.on_mouse_up(update_end)

In [ ]:
canvas

In [ ]:
debug_view

In [ ]:
a = Image.from_file("../data/projects/bbox/pics/test200x200.png")

In [ ]:
#exporti

def points2bbox_coords(start_x, start_y, end_x, end_y):
    min_x, max_x = sorted((start_x, end_x)) 
    min_y, max_y = sorted((start_y, end_y))
    return {'x': min_x, 'y': min_y, 'width': max_x-min_x, 'height': max_y-min_y}


# Create BBoxCanvas

![](http://www.plantuml.com/plantuml/png/bP5DQiCm48NtEiL0DmdK5yWY1DehT2t6qZWnbJL6FtPo-rfIxQIj25JWXVK-FTwyEVc0BiJ125I17NnuSl0oN_f0Gh4DZWsyePtG2o6Is1pBxm46Zfv0ysp5sN4SuTtXtDkpA41pxDnYG5OVWAtCjCpqUlz8o9n3wcyBmpn2nmu8rGL6xXVCmS0J2LXEBSWcbcjeZd2ttrrVuRRFCfe0FYf5tdVHW4AVdfLJInP73MtBLwcOdVOLTisHmcSqCsGy0SS0t85AnD-dMyQr0Xwt1ZQBc3QIfErvRVfubv_KNwZ_PR1QfAzMpOIcrnW8P1bGFlPKx_bvRtdWYTmD_836mCwoRQRUzwsXVK7g5ieGMlWLj4NrkQuG0_tOucaHuc2-0000)

In [ ]:
#export

class BBoxCanvas(HBox, traitlets.HasTraits):
    """
    Represents canvas holding image and bbox ontop. 
    Gives user an ability to draw a bbox with mouse.
    
    """
    debug_output = widgets.Output(layout={'border': '1px solid black'})
    image_path = traitlets.Unicode()
    bbox_coords = traitlets.Dict()
    _canvas_bbox_coords = traitlets.Dict()
    _image_scale = traitlets.Float()
    
    def __init__(self, width, height):
        super().__init__()       

        self.is_drawing = False
        self._start_point = ()
        self._image_scale = 1.0
        
        self._bg_layer = 0
        self._image_layer = 1
        self._box_layer = 2
        # do not stick bbox to borders
        self.padding = 2

        # Define each of the children...
        self._image = Image(layout=Layout(display='flex',
                                          justify_content='center',
                                          align_items='center',
                                          align_content='center'))
        self._multi_canvas = MultiCanvas(3, width=width, height=height)
        self._im_name_box = Label()
            
        children = [VBox([self._multi_canvas, self._im_name_box])]
        self.children = children
        draw_bg(self._multi_canvas[self._bg_layer])
        
        # link drawing events        
        self._multi_canvas[self._box_layer].on_mouse_move(self._update_pos)
        self._multi_canvas[self._box_layer].on_mouse_down(self._start_drawing)
        self._multi_canvas[self._box_layer].on_mouse_up(self._stop_drawing)
        
    
    @debug_output.capture(clear_output=False)
    def _update_pos(self, x, y):
        if self.is_drawing:
            self._canvas_bbox_coords = points2bbox_coords(*self._start_point, x, y)
            # bbox should not cross the canvas border:
            if self._invalid_coords(x, y):
                print(' !! Out of canvas border !!')
                self._stop_drawing(x, y)
                    
    def _invalid_coords(self, x, y):
        return (self._canvas_bbox_coords["x"] + self._canvas_bbox_coords["width"] > self._multi_canvas.width - self.padding or
                self._canvas_bbox_coords["y"] + self._canvas_bbox_coords["height"] > self._multi_canvas.height - self.padding or
                self._canvas_bbox_coords["x"] < self.padding or
                self._canvas_bbox_coords["y"] < self.padding)
    
        
    @debug_output.capture(clear_output=True)            
    def _start_drawing(self, x, y):
#         print("-> START DRAWING")
        self._start_point = (x, y)
        self.is_drawing = True
#         print("<- START DRAWING")

    @debug_output.capture(clear_output=False)
    def _stop_drawing(self, x, y):
#         print("-> STOP DRAWING")
        self.is_drawing = False
        
        # if something is drawn
        if self._canvas_bbox_coords:
            # if bbox is not human visible, clean:
            if (self._canvas_bbox_coords['width'] < 10 or 
                self._canvas_bbox_coords['height'] < 10):
                self._canvas_bbox_coords = {}
                print(" !! too small bbox drawn !!")
            else: # otherwise, save bbox values to backend
                self.bbox_coords = dict({ k: v / self._image_scale for k, v in self._canvas_bbox_coords.items() })
#         print("<- STOP DRAWING")
        
        
    @traitlets.observe('bbox_coords')
    def _update_canvas_bbox_coords(self, change):
#         print('-> Observe bbox_coords: ', change)
        
        if change['new'] == self._canvas_bbox_coords: # change event from gui, do nothing
            pass
#             print('-> GUI')
        else: # recalculate canvas coordinates as bbox was set by backend:
            self._canvas_bbox_coords = {k: v * self._image_scale for k, v in self.bbox_coords.items()}
#             print('-> Backend')
#         print('<- Observe bbox_coords')
               
    
    @traitlets.observe('_canvas_bbox_coords')
    def _draw_bbox(self, change):
#         print('-> Observe canvas_coords: ', change)
        if not self._canvas_bbox_coords:
            self._clear_bbox()
            self.bbox_coords = {}
            return
        coords = [self._canvas_bbox_coords['x'],
                  self._canvas_bbox_coords['y'],
                  self._canvas_bbox_coords['width'],
                  self._canvas_bbox_coords['height']]
        draw_bounding_box(self._multi_canvas[self._box_layer], coords,
                          color='white', border_ratio=2, clear=True)
#         print('<- Observe canvas_coords')
        
        
    def _clear_bbox(self):
        self._multi_canvas[self._box_layer].clear()
    
        
    @traitlets.observe('image_path')
    def _draw_image(self, image):
        self._image_scale = draw_img(self._multi_canvas[self._image_layer], self.image_path, clear=True)
        self._im_name_box.value = Path(self.image_path).name

    @property
    def image_scale(self):
        return self._image_scale
    
    def _clear_image(self):
        self._multi_canvas[self._image_layer].clear()
    
    # needed to support voila
    # https://ipycanvas.readthedocs.io/en/latest/advanced.html#ipycanvas-in-voila
    def observe_client_ready(self, cb=None):
        self._multi_canvas.on_client_ready(cb)

In [ ]:
gui = BBoxCanvas(width=100, height=100)
gui

In [ ]:
gui.debug_output

In [ ]:
gui.bbox_coords

In [ ]:
# gui.bbox_coords = {'x':5, 'y': 5, 'width': 5, 'height': 5}
gui.bbox_coords = {}

In [ ]:
# gui.image_path = "../data/projects/bbox/pics/red400x640.png"
gui.image_path = '../data/projects/im2im1/class_images/blocks_1.png'

In [ ]:
gui.image_scale

In [ ]:
# gui._draw_bbox([10, 20, 80, 100])
gui.bbox_coords = {'x': 5, 'y': 10, 'width': 20, 'height': 30}

In [ ]:
gui.bbox_coords = {'x': 0, 'y': 10, 'width': 20, 'height': 30}

In [ ]:
gui._clear_bbox()

# MultiBBoxCanvas

MultiBBoxCanvas allow drawing multiple bbox over a canvas.
There are 4 layers:
- background layer
- image layer
- validated bbox layer
- currently drawing bbox layer




In [ ]:
#export

class MultiBBoxCanvas(HBox, traitlets.HasTraits):
    """
    Represents canvas holding an image and multiple bbox ontop.
    Gives user an ability to draw multiple bbox with mouse.
    """
    debug_output = widgets.Output(layout={'border': '1px solid black'})
    image_path = traitlets.Unicode()
    # bbox_coords associates each bbox_index with bbox's coords
    bbox_coords = traitlets.Dict()
    # Current bbox_coords to process
    _canvas_bbox_coords = traitlets.Dict()
    _image_scale = traitlets.Float()

    def __init__(self, width, height):
        super().__init__()

        self.is_drawing = False
        self._start_point = ()
        self._image_scale = 1.0

        self._bg_layer = 0
        self._image_layer = 1
        # layer where validated bbox are drawn
        self._bbox_layer = 2
        # layer where bbox are drawn before being validated
        self._drawing_layer = 3
        self._total_layers = 4  # (bg + img + bbox + drawing)
        # do not stick bbox to borders
        self.padding = 2
        # Internal index for multiple bbox management
        self._bbox_index = 0

        # Define each of the children...
        self._image = Image(layout=Layout(display='flex',
                                          justify_content='center',
                                          align_items='center',
                                          align_content='center'))
        self._multi_canvas = MultiCanvas(self._total_layers,
                                         width=width,
                                         height=height)
        self._im_name_box = Label()

        children = [VBox([self._multi_canvas, self._im_name_box])]
        self.children = children
        draw_bg(self._multi_canvas[self._bg_layer])

        # link drawing events
        self._multi_canvas[self._drawing_layer].on_mouse_move(self._update_pos)
        self._multi_canvas[self._drawing_layer].on_mouse_down(self._start_drawing)
        self._multi_canvas[self._drawing_layer].on_mouse_up(self._stop_drawing)

    @debug_output.capture(clear_output=True)
    def _start_drawing(self, x, y):
        if not self.is_drawing:  # protect against right clicking while drawing
            self._start_point = (x, y)
            self.is_drawing = True

    @debug_output.capture(clear_output=False)
    def _update_pos(self, x, y):
        if self.is_drawing:
            self._canvas_bbox_coords = points2bbox_coords(*self._start_point, x, y)
            print(f"active bbox coords: {self._canvas_bbox_coords}")
            # bbox should not cross the canvas border:
            if self._invalid_coords(x, y):
                print(f" !! bbox Out of canvas border !!")
                self._stop_drawing(x, y)

    def _invalid_coords(self, x, y):
        return (self._canvas_bbox_coords["x"] + self._canvas_bbox_coords["width"] > self._multi_canvas.width - self.padding or
                self._canvas_bbox_coords["y"] + self._canvas_bbox_coords["height"] > self._multi_canvas.height - self.padding or
                self._canvas_bbox_coords["x"] < self.padding or
                self._canvas_bbox_coords["y"] < self.padding)

    @debug_output.capture(clear_output=False)
    def _stop_drawing(self, x, y):
        self.is_drawing = False
        # if something is drawn
        if self._canvas_bbox_coords:
            # if bbox is not human visible, clean:
            if (self._canvas_bbox_coords['width'] < 10 or
                    self._canvas_bbox_coords['height'] < 10):
                self._canvas_bbox_coords = {}
                print(" !! bbox too small to be drawn !!")
            else:  # otherwise, save bbox values to backend
                # We use a copy to force traitlets.observe to register the change
                tmp_bbox_coords = deepcopy(self.bbox_coords)
                self._bbox_index += 1
                tmp_bbox_coords[self._bbox_index] = dict({k: v / self._image_scale for k, v in self._canvas_bbox_coords.items()})
                self.bbox_coords = tmp_bbox_coords
  
    @traitlets.observe('bbox_coords')
    def _update_canvas_bbox_coords(self, change):
        if len(self.bbox_coords.keys()) > 0:
            self._bbox_index = int(list(sorted(self.bbox_coords.keys()))[-1])
        else:
            self._bbox_index = 0
        self._draw_all_bbox()

    def _draw_all_bbox(self):
        self._canvas_bbox_coords = {}
        # Redraw the whole bbox layer
        self._clear_layer(self._bbox_layer)
        for index, data in self.bbox_coords.items():
            coords = [data['x'],
                      data['y'],
                      data['width'],
                      data['height']]
            coords = [value * self._image_scale for value in coords]
            draw_bounding_box(self._multi_canvas[self._bbox_layer], coords,
                              color='white', border_ratio=2, clear=False)

    @traitlets.observe('_canvas_bbox_coords')
    def _draw_bbox(self, change):
        if not self._canvas_bbox_coords:
            self._clear_layer(self._drawing_layer)
            return
        coords = [self._canvas_bbox_coords['x'],
                  self._canvas_bbox_coords['y'],
                  self._canvas_bbox_coords['width'],
                  self._canvas_bbox_coords['height']]
        draw_bounding_box(self._multi_canvas[self._drawing_layer], coords,
                          color='white', border_ratio=2, clear=True)

    def _clear_layer(self, index):
            self._multi_canvas[index].clear()

    def clear_all_bbox(self):
        self._clear_layer(self._bbox_layer)
        self._clear_layer(self._drawing_layer)
        self.bbox_coords = {}

    @debug_output.capture(clear_output=False)
    def clear_last_bbox(self):
        if self.bbox_coords:
            # We use a copy to force traitlets.observe to register the change
            tmp_bbox_coords = deepcopy(self.bbox_coords)
            removed_bbox = tmp_bbox_coords.popitem()
            self.bbox_coords = tmp_bbox_coords
            print(f"bbox {removed_bbox} deleted")

    @traitlets.observe('image_path')
    def _draw_image(self, image):
        self._image_scale = draw_img(self._multi_canvas[self._image_layer], self.image_path, clear=True)
        self._im_name_box.value = Path(self.image_path).name

    @property
    def image_scale(self):
        return self._image_scale

    def _clear_image(self):
        self._clear_layer(self._image_layer)

    # needed to support voila
    # https://ipycanvas.readthedocs.io/en/latest/advanced.html#ipycanvas-in-voila
    def observe_client_ready(self, cb=None):
        self._multi_canvas.on_client_ready(cb)


In [ ]:
gui = MultiBBoxCanvas(width=500, height=500)
gui

In [ ]:
gui.image_path = '../data/projects/im2im1/class_images/blocks_1.png'

In [ ]:
gui.debug_output

In [ ]:
gui.bbox_coords

In [ ]:
gui.bbox_coords={1: {'x': 2, 'y': 12, 'width': 7, 'height': 8},
                 95: {'x': 15, 'y': 15, 'width': 15, 'height': 10},
                 50: {'x': 40, 'y': 40, 'width': 6, 'height': 10},
                 100: {'x': 30, 'y': 25, 'width': 10, 'height': 15}}

In [ ]:
gui.clear_last_bbox()
gui.clear_last_bbox()
assert gui.bbox_coords == {1: {'x': 2, 'y': 12, 'width': 7, 'height': 8},
                 95: {'x': 15, 'y': 15, 'width': 15, 'height': 10}}

In [ ]:
gui.clear_all_bbox()
assert gui.bbox_coords == {}

In [ ]:
#hide
from nbdev.export import *
notebook2script()